In [1]:
import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

from keras.layers.normalization import BatchNormalization
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image

from numpy import genfromtxt

from keras import callbacks

import pandas as pd
from sklearn.metrics import mean_squared_error

from tqdm import tqdm

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5110)
/opt/conda/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
path = "../../sudoku/sudoku.csv"

In [56]:
sudoku_games = genfromtxt(path,dtype='string', delimiter=',',skip_header=1)[:110000]
sudoku_games.shape

(110000, 2)

In [57]:
train_data = sudoku_games[:,0]

train_data_list = []

for data in tqdm(train_data):
    train_data_list.append(np.array(list(data)).reshape((9,9)))
    
train_data = np.stack(train_data_list)
train_data = np.expand_dims(train_data,axis=1)

100%|██████████| 110000/110000 [00:00<00:00, 153560.70it/s]


In [58]:
train_labels = sudoku_games[:,1]

train_label_list = []

for data in tqdm(train_labels):
    train_label_list.append(np.array(list(data)).reshape((9,9)))
    
train_labels = np.stack(train_label_list)
train_labels = np.expand_dims(train_labels,axis=1)

100%|██████████| 110000/110000 [00:00<00:00, 150141.31it/s]


In [59]:
indexed_train_labels = np.zeros((train_labels.shape[0],10,9,9))
indexed_train_labels.shape

for i,train_label in tqdm(enumerate(train_labels)):
    train_label = np.squeeze(train_label)
    for row in range(9):
        for column in range(9):
            value = int(train_label[row,column])
            indexed_train_labels[i,value,row,column] = 1

110000it [00:08, 12685.55it/s]


In [60]:
test_data = train_data[100000:]
test_labels_indexed = indexed_train_labels[100000:]

train_data = train_data[:100000]
train_labels_indexed = indexed_train_labels[:100000]


In [61]:
print(train_data.shape)
print(train_labels_indexed.shape)

print(test_data.shape)
print(test_labels_indexed.shape)

(100000, 1, 9, 9)
(100000, 10, 9, 9)
(10000, 1, 9, 9)
(10000, 10, 9, 9)


In [62]:
train_data[0]

array([[['0', '0', '4', '3', '0', '0', '2', '0', '9'],
        ['0', '0', '5', '0', '0', '9', '0', '0', '1'],
        ['0', '7', '0', '0', '6', '0', '0', '4', '3'],
        ['0', '0', '6', '0', '0', '2', '0', '8', '7'],
        ['1', '9', '0', '0', '0', '7', '4', '0', '0'],
        ['0', '5', '0', '0', '8', '3', '0', '0', '0'],
        ['6', '0', '0', '0', '0', '0', '1', '0', '5'],
        ['0', '0', '3', '5', '0', '8', '6', '9', '0'],
        ['0', '4', '2', '9', '1', '0', '3', '0', '0']]], 
      dtype='|S1')

In [63]:
train_labels[0]

array([[['8', '6', '4', '3', '7', '1', '2', '5', '9'],
        ['3', '2', '5', '8', '4', '9', '7', '6', '1'],
        ['9', '7', '1', '2', '6', '5', '8', '4', '3'],
        ['4', '3', '6', '1', '9', '2', '5', '8', '7'],
        ['1', '9', '8', '6', '5', '7', '4', '3', '2'],
        ['2', '5', '7', '4', '8', '3', '9', '1', '6'],
        ['6', '8', '9', '7', '3', '4', '1', '2', '5'],
        ['7', '1', '3', '5', '2', '8', '6', '9', '4'],
        ['5', '4', '2', '9', '1', '6', '3', '7', '8']]], 
      dtype='|S1')

In [64]:
model = Sequential()
model.add(Convolution2D(64, 3, 3, activation='relu',border_mode='same', input_shape=(1, 9, 9)))
model.add(Convolution2D(64, 3, 3, activation='relu',border_mode='same'))
model.add(Convolution2D(64, 3, 3, activation='relu',border_mode='same'))
model.add(Convolution2D(64, 3, 3, activation='relu',border_mode='same'))
model.add(Convolution2D(64, 3, 3, activation='relu',border_mode='same'))
model.add(Convolution2D(64, 3, 3, activation='relu',border_mode='same'))
model.add(Convolution2D(10, 1, 1, activation='sigmoid'))

model.compile(optimizer=Adam(lr=0.001),
                loss='mse')

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_53 (Convolution2D) (None, 64, 9, 9)      640         convolution2d_input_9[0][0]      
____________________________________________________________________________________________________
convolution2d_54 (Convolution2D) (None, 64, 9, 9)      36928       convolution2d_53[0][0]           
____________________________________________________________________________________________________
convolution2d_55 (Convolution2D) (None, 64, 9, 9)      36928       convolution2d_54[0][0]           
____________________________________________________________________________________________________
convolution2d_56 (Convolution2D) (None, 64, 9, 9)      36928       convolution2d_55[0][0]           
___________________________________________________________________________________________

In [66]:
model.fit(train_data, 
          train_labels_indexed, 
          validation_data = (test_data,test_labels_indexed),
          batch_size=1024, 
          callbacks = [callbacks.CSVLogger("./training.log")],
          nb_epoch=5)

Train on 100000 samples, validate on 10000 samples
Epoch 1/5
100000/100000 [==============================] - 7s - loss: 0.1031 - val_loss: 0.1000

KeyboardInterrupt: 

In [ ]:
model.save_weights("sudoku-model-indexed.h5")

# Preds

In [ ]:
raw_train_preds = model.predict(train_data)
raw_train_preds.shape

In [ ]:
mean_squared_error(train_labels.flatten().astype(float), raw_train_preds.flatten().astype(float))  

In [ ]:
rounded_train_preds = np.round(raw_train_preds)

In [ ]:
mean_squared_error(train_labels.flatten().astype(float), rounded_train_preds.flatten().astype(float))  

In [ ]:
test_example = np.expand_dims(test_data[0],axis=0)

In [ ]:
test_example.shape

In [ ]:
predicted_data = model.predict(test_example)

In [ ]:
predicted_data

In [ ]:
test_labels[0]

In [ ]:
np.round(predicted_data)

In [ ]:
test_data[0]